# Homework 6

**Due: 11/8/2020 on gradescope**

## References

+ Lectures 9-11 (inclusive).

## Instructions

+ Type your name and email in the "Student details" section below.
+ Develop the code and generate the figures you need to solve the problems using this notebook.
+ For the answers that require a mathematical proof or derivation you can either:
    
    - Type the answer using the built-in latex capabilities. In this case, simply export the notebook as a pdf and upload it on gradescope; or
    - You can print the notebook (after you are done with all the code), write your answers by hand, scan, turn your response to a single pdf, and upload on gradescope.

+ The total homework points are 100. Please note that the problems are not weighed equally.

**Note**: Please match all the pages corresponding to each of the questions when you submit on gradescope. 

## Student details

+ **First Name:**
+ **Last Name:**
+ **Email:**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')
import scipy.stats as st
# A helper function for downloading files
import requests
import os
def download(url, local_filename=None):
    """
    Downloads the file in the ``url`` and saves it in the current working directory.
    """
    data = requests.get(url)
    if local_filename is None:
        local_filename = os.path.basename(url)
    with open(local_filename, 'wb') as fd:
        fd.write(data.content)

# Problem 1 - Estimating the mechanical properties of a plastic material from molecular dynamics simulations

First, make sure that [this](https://raw.githubusercontent.com/PredictiveScienceLab/data-analytics-se/master/homework/stress_strain.txt) dataset is visible from this Jupyter notebook.
You may achieve this by either:

+ Downloading the data file, putting it in your Google drive, mounting the drive, and changing to the directory of the file (see Problem 0 in [Homework](https://colab.research.google.com/github/PredictiveScienceLab/data-analytics-se/blob/master/homework/homework_03.ipynb); or
+ Downloading the file to the working directory of this notebook with this code:

In [ ]:
url = 'https://raw.githubusercontent.com/PurdueMechanicalEngineering/me-297-intro-to-data-science/master/homework/stress_strain.txt'
download(url)

It's up to you what you choose to do.
If the file is in the right place, the following code should work:

In [ ]:
data =  np.loadtxt('stress_strain.txt')

The dataset was generated using a molecular dynamics simulation of a plastic material (thanks to [Professor Alejandro Strachan](https://engineering.purdue.edu/MSE/people/ptProfile?id=33239) for sharing the data!).
Specifically, Strachan's group did the following:
- They took a rectangular chunk of the material and marked the position of each one of its atoms;
- They started applying a tensile force along one dimension.
The atoms are coupled together through electromagnetic forces and they must all satisfy Newton's law of motion.
- For each value of the applied tensile force they marked the stress (force be unit area) in the middle of the materail and the corresponding strain of the material (percent enlogation in the pulling direction).
- Eventually the material entered the plastic regime and then it broke.
Here is a visualization of the data:

In [ ]:
x = data[:, 0] # Strain 
y = data[:, 1] # Stress in MPa
plt.figure()
plt.plot(x, y, 'ro', markersize=2, label = 'Stress-strain data')
plt.xlabel('$\epsilon$ (strain in %)', fontsize=14)
plt.ylabel('$\sigma$ (stress in MPa)', fontsize=14)
plt.legend(loc='best', fontsize = 14);

Note that for each particular value of the strain, you don't necessarily get a unique stress.
This is because in molecular dynamics the atoms are jiggling around due to thermal effects.
So there is always this "jiggling" noise when you are trying to measure the stress and the strain.
We would like to process this noise in order to extract what is known as the [stress-strain curve](https://en.wikipedia.org/wiki/Stress–strain_curve) of the material.
The stress-strain curve is a macroscopic property of the the material which is affeted by the fine structure, e.g., the chemical bonds, the crystaline structure, any defects, etc.
It is a required input to mechanics of materials.

## Part A - Fitting the stress-strain curve in the elastic regime
The very first part of the stress-strain curve should be linear.
It is called the *elastic regime*.
In that region, say $\epsilon < \epsilon_l=0.04$, the relationship between stress and strain is:
$$
\sigma(\epsilon) = E\epsilon.
$$
The constant $E$ is known as the *Young modulus* of the material.
Assume that you measure $\epsilon$ without any noise, but your measured $\sigma$ is noisy.

First, extract the relevant data for this problem, split it into training and validation datasets, and visualize the training and validation datasets using different colors.

In [ ]:
# The point at which the stress-strain curve stops being linear
epsilon_l = 0.04
# Relevant data (this is nice way to get the linear part of the stresses and straints)
x_rel = x[x < 0.04]
y_rel = y[x < 0.04]
# Visualize to make sure you have the right data
plt.figure()
plt.plot(x_rel, y_rel, 'ro', markersize=2, label = 'Stress-strain data')
plt.xlabel('$\epsilon$ (strain in %)', fontsize=14)
plt.ylabel('$\sigma$ (stress in MPa)', fontsize=14)
plt.legend(loc='best', fontsize = 14);

We are going to start by splitting the data into training and validation sets. We will use the validation set to test how good our model is.

In [ ]:
# Split the data into training and validation datasets
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_rel, y_rel, test_size=0.33)

### Subpart A.I

In the same plot, show both the training and the validation data.
Make sure you name the axes properly.

In [ ]:
# your code here

### Subpart A.II

Perform Bayesian linear regression using [scikit.learn.BayesianRidge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html).
Note that this kind of regression model estimates the measurement noise as well as the hyper-parameter for the weights for you. You don't have to pick it by hand. It does this through a process called the evidence approximation. The details of the evidence approximation are rather advanced and not covered in the class.
However, BayesianRidge works quite well as an out of the box Bayesian regression model.

In [ ]:
# We just need the design matrix.
# We know that the constant should be zero because of the physics of the problem.
# So, let's force it to be zero by NOT including the constant term in our model
Phi_train = # Write code do define the design matrix. It should be N x 1
# Performing Bayesian linear regression using scikit learn
from sklearn.linear_model import BayesianRidge
# Below is how you initialize the train the BayesianRidge model
# You need to give it the design matrix and the output data
# normalize=True simply scales the input and the output (strains and stresses) before fitting
# (by subtracting the mean and dividing by the standard deviation)
# Note that if you don't specify the fit_intercept=False below, BayesianRidge will
# automatically add a constant offset to your model (which here should be zero)
model = BayesianRidge(fit_intercept=False).fit(Phi_train, y_train)
# The notation followed by BayesianRidge is different than the notation 
# we used in the class for Bayesian regression
# In particular, what we called sigma (the square root of measurement noise variance)
# they call 1 / model.alpha_
# And what we called alpha (the regularization parameter for the weights),
# they call 1 / model.lambda_
# Here is how you can get the square root of the measurement noise variance
sigma = np.sqrt(1.0 / model.alpha_)
print('sigma = {0:1.2f}'.format(sigma))
# Here is how you can get regularization parameter:
alpha = np.sqrt(1.0 / model.lambda_)
print('alpha = {0:1.2f}'.format(alpha))

### Subpart A.III
Calculate the mean square error of the validation data.

In [ ]:
# Here is how you can make predictions using the model:
y_valid_p_mean = model.predict(x_valid[:, None])
MSE = np.mean((y_valid_p_mean - y_valid) ** 2)
print('MSE = {0:1.2f}'.format(MSE))

### Subpart A.IV
Make the observations vs predictions plot for the validation data.

In [ ]:
# Your code here

### Subpart A.V
Visualize your epistemic and the aleatory uncertainty about the stress-strain curve in the elastic regime.

In [ ]:
# Get some strain points
xx = np.linspace(0.0, 0.05, 100)
# Use the model to get the predictive mean and standard deviation:
yy_mean, yy_measured_std = model.predict(xx[:, None], return_std=True)
# Separate the epistemic uncertainty
yy_std = np.sqrt(yy_measured_std ** 2 - sigma**2)
# Your code here

### Subpart A. VI
Visualize the posterior of the Young modulus E conditioned on the data.

In [ ]:
# The posterior mean of the weights is here
m_norm = model.coef_
print(m_norm)
# The posterior covariance matrix for the weights is here 
S_norm = model.sigma_
print(S_norm)
# Create a Normal random variable to represent this
E_post = st.norm(loc=m_norm[0], scale=np.sqrt(S_norm[0, 0]))
# Plot the PDF of E_post
# Your code here

### Subpart A.VII
Take five samples of stress-strain curve in the elastic regime and visualize them.

In [ ]:
fig, ax = plt.subplots(dpi=100)
for i in range(5):
    E = # your code here - take the sample
    yy_sample = E * xx
    ax.plot(xx, yy_sample, 'r', lw=0.5)
# plot the data again
ax.plot(x_train, y_train, 'kx', label='Observed data')
plt.xlabel('$\epsilon$ (strain in %)', fontsize=14)
plt.ylabel('$\sigma$ (stress in MPa)', fontsize=14)
plt.legend(loc='best', fontsize = 14);

### Subpart A.VIII

Find the 95% centered credible interval for the Young modulus $E$.

In [ ]:
# Your code here

### Subpart A.IX
If you had to pick a single value for the Young modulus $E$, what would it be and why?

In [ ]:
# Your code here

This is a bit subjective, but I would pick the median of the distribution (which here happens to be the same as the max of the posterior and the mean).

## Part B - Estimate the ultimate strength

The pick of the stress-strain curve is known as the ultimate strength.
We will like to estimate it.

### Subpart B.I - Extract training and validation data

Extract training and validation data from the **entire** dataset.

In [ ]:
# Your code here

Use the following to visualize your split:

In [ ]:
# Your code here

### Subpart B.II - Model the entire stress-strain relationship.
To do this, we will set up a generalized linear model that can capture the entire stress-strain relationship.
Remember, you can use any model you want as soon as:
+ it is linear in the parameters to be estimated,
+ it clearly has a well-defined elastic regime (see Part A).

I am going to help you set up the right model.
We are goint to use the [Heavide step function](https://en.wikipedia.org/wiki/Heaviside_step_function) to turn on or off models for various ranges of $\epsilon$. The idea is quite simple: We will use a linear model for the elastic regime and we are going to turn to a non-linear model for the non-linear regime.
Here is a model that has the right form in the elastic regime and an arbitrary form in the non-linear regime:
$$
f(\epsilon;E,\mathbf{w}_g) = E\epsilon \left[(1 - H(\epsilon - \epsilon_l)\right] + g(\epsilon;\mathbf{w}_g)H(\epsilon - \epsilon_l),
$$
where
$$
H(x) = \begin{cases}
0,\;\text{if}\;x < 0\\
1,\;\text{otherwise},
\end{cases}
$$
and $g$ is any function linear in the parameters $\mathbf{w}_g$.

You can use any model you like for the non-linear regime, but let's use a polynomial of degree $d$:
$$
g(\epsilon) = \sum_{i=0}^{d} w_i \epsilon^i.
$$

The full model can be expressed as:
$$
\begin{split}
f(\epsilon) &= 
\begin{cases}
h(\epsilon) = E \epsilon,\ \epsilon < \epsilon_l, \\
g(\epsilon) = \sum_{i=0}^{d} w_i \epsilon^i, \epsilon \geq \epsilon_l
\end{cases}\\
&= E\epsilon \left(1 - H(\epsilon - \epsilon_l)\right) + \sum_{i=0}^{d} w_i \epsilon^iH(\epsilon - \epsilon_l).
\end{split}
$$
We could proceed with this model, but there is a small problem: It is discontinuous at $\epsilon = \epsilon_l$.
This is unphysical. We can do better than that!

To make the model nice, we force the $h$ and $g$ to match up to the first derivative, i.e., we demand that:
$$
\begin{split}
h(\epsilon_l) &= g(\epsilon_l)\\
h'(\epsilon_l) &= g'(\epsilon_l).
\end{split}
$$
The reason we include the first derivative is so that we don't have a kink in the stress-strain. That would also be unphysical.
The two equations above become:
$$
\begin{split}
E\epsilon_l &= \sum_{i=0}^dw_i\epsilon_l^i\\
E &= \sum_{i=1}^diw_i\epsilon_l^{i-1}.
\end{split}
$$
We can use these two equations to eliminate two weights.
Let's eliminate $w_0$ and $w_1$.
All you have to do is express them in terms of $E$ and $w_2,\dots,w_d$.
So, there remain $d$ parameters to estimate.
Let's get back to the stress-strain model.

Our stress-strain model was:
$$
f(\epsilon) = E\epsilon \left(1 - H(\epsilon - \epsilon_l)\right) + \sum_{i=0}^{d} w_i \epsilon^iH(\epsilon - \epsilon_l).
$$
We can now use the expressions for $w_0$ and $w_1$ to rewrite this using only all the other parameters.
I am going to spare you the details...
The end result is:
$$
f(\epsilon) = E\epsilon + \sum_{i=2}^dw_i\left[(i-1)\epsilon_{l}^{i} - i \epsilon \epsilon_{l}^{i-1} + \epsilon^i\right]H(\epsilon - \epsilon_l).
$$
Okay.
This is still a generalized linear model. This is nice.
Write code for the design matrix:

In [ ]:
# Complete this code to make your model:
def compute_design_matrix(Epsilon, epsilon_l, d):
    """
    Computes the design matrix for the stress-strain curve problem.
    
    Arguments:
        Epsilon     -     A 1D array of dimension N.
        epsilon_l   -     The strain signifying the end of the elastic regime.
        d           -     The polynomial degree.
    
    Returns:
        A design matrix N x d
    """
    # Sanity check
    assert isinstance(Epsilon, np.ndarray)
    assert Epsilon.ndim == 1, 'Pass the array as epsilon.flatten(), if it is two dimensional'
    n = Epsilon.shape[0]
    # The design matrix:
    Phi = np.ndarray((n, d))
    # The step function
    Step = np.ones(n)
    Step[Epsilon < epsilon_l] = 0
    # Build the design matrix
    Phi[:, 0] = # Your code here
    for i in range(2, d+1):
        Phi[:, i-1] = # Your code here
    return Phi

Visualize the basis functions here:

In [ ]:
d = 4
eps = np.linspace(0, x.max(), 100)
Phis = compute_design_matrix(eps, epsilon_l, d)
fig, ax = plt.subplots(dpi=100)
ax.plot(eps, Phis)
ax.set_xlabel('$\epsilon$ (strain in %))')
ax.set_ylabel('$\phi_i(\epsilon)$');

### Subpart B.III 

Fit the model using Bayesian regression and demonstrate that it works well by doing all the things we did above (MSE, observations vs predictions plot, standarized errors, etc.).

In [ ]:
Phi_train = compute_design_matrix(x_train, epsilon_l, d)
model = # Your code here using BayesianRidge

Let's calculate the MSE:

In [ ]:
# Your code here

Make the observations vs predictions plot:

In [ ]:
# Your code here

### Subpart B.IV
Visualize epistemic and aleatory uncertainty in the stess-strain relation.

In [ ]:
# Your code here

### Subpart B.V - Extract the ultimate strength

Now, you are going to quantify your epistemic uncertainty about the ultimate strength.
The ultimate strength is the maximum of the stress-strain relationship.
Since you have epistemic uncertainty about the stress-strain relationship, you also have epistemic uncertainty about the ultimate strength.

Do the following:
- Visualize posterior of the ultimate strength.
- Find a 95% credible interval for the ultimate strength.
- Pick a value for the ultimate strength.

**Hint:**
To characterize your epistemic uncertainty about the ultimate strength, you would have to do the following:
- Define a dense set of strain points between 0 and 0.25.
- Repeatedly:
    + sample from the posterior of the weights of your model
    + for each sample evaluate the stresses at the dense set of strain points defined earlier
    + for each sampled stress vector, find the maximum. This is a sample of the ultimate strength.

In [ ]:
# The posterior mean of the weights is here (this is for the normalized features, however)
m_norm = model.coef_
# The posterior covariance matrix for the weights is here (also for the normalized features)
S_norm = model.sigma_ + 1e-6 * np.eye(model.sigma_.shape[0])
# The posterior of the weights
w_post = st.multivariate_normal(mean=m_norm, cov=S_norm)
# The number of samples to take
num_post_samples = 1000
# Storing the ultimate strength of each one of the posterior samples
ultimate_strength_post_samples = np.ndarray((num_post_samples,))
# Start taking samples
for n in range(num_post_samples):
    w_sample = # Your code here
    yy_sample = # Your code here
    ultimate_strength_post_samples[n] = np.max(yy_sample)
# Do the histogram of ultimate_strength_post_samples
# Your code here

# Problem 2 - Optimizing the performance of a compressor

In this problem we are going to need [this](https://raw.githubusercontent.com/PredictiveScienceLab/data-analytics-se/master/homework/compressor_data.xlsx) dataset. The dataset was kindly provided to us by [Professor Davide Ziviani](https://scholar.google.com/citations?user=gPdAtg0AAAAJ&hl=en).
As before, you can either put it on your Google drive or just download it with the code segment below:

In [ ]:
url = 'https://raw.githubusercontent.com/PurdueMechanicalEngineering/me-297-intro-to-data-science/master/homework/compressor_data.xlsx'
download(url)

Note that this is an Excell file, so we are going to need pandas to read it.
Here is how:

In [ ]:
import pandas as pd
data = pd.read_excel('compressor_data.xlsx')
data

The data are part of a an experimental study of a variable speed reciprocating compressor.
The experimentalists varied two temperatures $T_e$ and $T_c$ (both in C) and they measured various other quantities.
Our goal is to learn the map between $T_e$ and $T_c$ and measured Capacity and Power (both in W).
First, let's see how you can extract only the relevant data.

In [ ]:
# Here is how to extract the T_e and T_c columns and put them in a single numpy array
X = data[['T_e','T_c']].values
X

In [ ]:
# Here is how to extract the Capacity
y = data['Capacity'].values
y

Fit the following multivariate polynomial model to **both the Capacity and the Power**:
$$
y = w_1 + w_2T_e + w_3 T_c + w_4 T_eT_c + w_5 T_e^2 + w_6T_c^2 + w_7 T_e^2T_c + w_8T_eT_c^2 + w_9 T_e^3 + w_{10}T_c^3 + \epsilon,
$$
where $\epsilon$ is a Gaussian noise term with unknown variance.
**Hints:**
+ You may use [sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to construct the design matrix of your polynomial features. Do not program the design matrix by hand.
+ You should split your data into training and validation and use various validation metrics to make sure that your models make sense.

## Part A - The Capacity

### Subpart A.I - Fit the capacity

Please don't just fit blindly. Split in training and test and use all the usual diagnostics.

In [ ]:
# For polynomial features
from sklearn.preprocessing import PolynomialFeatures
# For splitting the data
from sklearn.model_selection import train_test_split
# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33)
# Here is how to make Polynomials features
poly = PolynomialFeatures(degree=3)
# Design matrix for train data
Phi_train = poly.fit_transform(X_train)
# Fit with Bayesian Ridge - Use normalize=True to let BayesianRidge
# scale the inputs and outputs to reasonable values (it will subtract their empirical mean
# and divide by their empirical standard deviation)
# Your code here

The mean square error:

In [ ]:
# Your code here

The observations vs predictions plot.

In [ ]:
# Your code here

### Subpart A.II

What is the noise standard deviation you estimated for the Capacity?

In [ ]:
sigma = np.sqrt(1.0 / model.alpha_)
print('sigma = {0:1.2f} W'.format(sigma))

### Subpart B.I - Fit the Power

Please don't just fit blindly. Split in training and test and use all the usual diagnostics.

In [ ]:
# Here is how to extract the Capacity
y = data['Power'].values
y

In [ ]:
# Your code here

The mean square error:

In [ ]:
# Your code here

The observations vs predictions plot.

In [ ]:
# Your code here

### Subpart B.II

What is the noise standard deviation you estimated for the Power?

In [ ]:
# Your code here

# Problem 3 - Explaining the challenger disaster
On January 28, 1986, the [Space Shuttle Challenger](https://en.wikipedia.org/wiki/Space_Shuttle_Challenger_disaster) disintegrated after 73 seconds from launch.
The failure can be traced on the rubber O-rings which were used to seal the joints of the solid rocket boosters (required to force the hot, high-pressure gases generated by the burning solid propelant through the nozzles thus producing thrust).

It turns out that the performance of the O-ring material was particularly sensitive on the external temperature during launch.
This [dataset](https://raw.githubusercontent.com/PredictiveScienceLab/data-analytics-se/master/homework/challenger_data.csv) contains records of different experiments with O-rings recorded at various times between 1981 and 1986.
Download the data the usual way (either put them on Google drive or run the code cell below).

In [ ]:
url = 'https://raw.githubusercontent.com/PurdueMechanicalEngineering/me-297-intro-to-data-science/master/homework/challenger_data.csv'
download(url)

Even though this is a csv file, you should load it with pandas because it contains some special characters.

In [ ]:
raw_data = pd.read_csv('challenger_data.csv')
raw_data

The first column is the date of the record. The second column is the external temperature of that day in degrees F.
The third column labeled ``Damage Incident`` is has a binary coding (0=no damage, 1=damage).
The very last row is the day of the Challenger accident.

We are going to use the first 23 rows to solve a binary classification problem that will give us the probability of an accident conditioned on the observed external temperature in degrees F. Before we proceed to the analysis of the data, let's clean the data up.

First, we drop all the bad records:

In [ ]:
# Your code here

We also don't need the last record. Just remember that the temperature the day of the Challenger accident was 31 degrees F.
Remove the last record from the dataframe.

In [ ]:
# Your code here
clean_data = 

Let's extract the features and the labels:

In [ ]:
x = clean_data['Temperature'].values
x

In [ ]:
y = clean_data['Damage Incident'].values.astype(np.int)
y

## Part A - Perform logistic regression

Perform logistic regression between the temperature ($x$) and the damage label ($y$).
Do not bother doing a validation because there are not a lot of data.
Just use a very simple model so that you don't overfit.

**Answer:**
This is one of the cases, where we don't have a lot of data.
So we are going to use everything for training.
To avoid overfitting, we will use the simplest possible model.
The model is:
$$
p(y|x,w) = \operatorname{sigm}(w_0 + w_1 x),
$$
where $w_0$ and $w_1$ are parameters to be determined by data.

In [ ]:
# Your code here

This is it. Let's take a look at the parameters that that were found:

In [ ]:
# Your code here

We observe a negative correlation between temperature and damage. Damage becomes more probable as temperature decreases.

## Part B - Plot the probability of damage as a function of temperature
Plot the probability of damage as a function of temperature.

In [ ]:
# Your code here

## Part C - Decide whether or not to launch

The temperature the day of the Challenger accident was 31 degrees F.
Start by calculating the probability of damage at 31 degrees F.
Then, use formal decision-making (i.e., define a cost matrix and make decisions by minimizing the expected loss) to decide whether or not to launch on that day.
Also, plot your optimal decision as a function of the external temperature.

*Your answer here. As many code and text blocks as you need.* 